Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [30]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
import heapq 
from typing import Tuple, List, Optional, Union

In [31]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [32]:
#size: 10,20,50,100,200,500,1000
#density: 0.2,  0.5, 0.8, 1.0
#noise_level: 0, 0.1, 0.5, 0.8
#negative_values: True, False
problem = create_problem(1000, density=0.005, noise_level=10, negative_values=False)

In [33]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [34]:
def solver(problem: np.ndarray, source: int, target: int) -> Tuple[Optional[List[int]], Union[float, int]]:
    """
    Trova il percorso più breve usando l'algoritmo SPFA (Shortest Path Faster Algorithm),
    un'ottimizzazione di Bellman-Ford che usa una coda.
    Gestisce i pesi negativi e rileva i cicli negativi.
    """
    num_nodes = len(problem)
    
    dist = {node: np.inf for node in range(num_nodes)}
    parent = {node: None for node in range(num_nodes)}
    # Conta quante volte un nodo è stato rilassato
    relax_count = {node: 0 for node in range(num_nodes)}

    dist[source] = 0
    
    # Coda di priorità (min-heap) basata sul costo del percorso (g)
    # Come Dijkstra/Uniform-Cost Search [cite: 553]
    pq = [(0, source)]  # (g(source), source)
    
    while pq:
        # Estrai il nodo con il g_score più basso
        current_g, current_node = heapq.heappop(pq)
        
        # Ottimizzazione: se abbiamo trovato un percorso più breve per questo nodo
        # *dopo* averlo aggiunto alla coda, questa voce è "vecchia". Saltiamola.
        if current_g > dist[current_node]:
            continue
            
        # Rilassamento degli archi (come Bellman-Ford)
        for neighbor in range(num_nodes):
            cost = problem[current_node, neighbor]
            
            # Se esiste un arco (costo non infinito)
            if cost != np.inf:
                new_dist_g = current_g + cost
                
                # Trovato un percorso più breve?
                if new_dist_g < dist[neighbor]:
                    # *** CORREZIONE 1: Aggiorna la distanza ***
                    dist[neighbor] = new_dist_g
                    parent[neighbor] = current_node
                    
                    # Controllo Ciclo Negativo (come SPFA)
                    relax_count[neighbor] += 1
                    
                    # *** CORREZIONE 2: Controllo del ciclo corretto ***
                    # Un percorso semplice ha al massimo V-1 archi.
                    # Se un nodo viene rilassato V volte, abbiamo un ciclo negativo.
                    if relax_count[neighbor] >= num_nodes:
                        # Rilevato un ciclo negativo!
                        return None, -np.inf
                    
                    # Ri-aggiungi alla coda (con g_score come priorità)
                    heapq.heappush(pq, (new_dist_g, neighbor))

    # --- Ricostruzione del percorso (dopo il ciclo while) ---
    
    # *** CORREZIONE 3: Logica di ricostruzione del percorso completa ***
    if dist[target] == np.inf:
        # Nessun percorso trovato
        return None, np.inf  
    
    path = []
    curr = target
    while curr is not None:
        path.append(curr)
        if curr == source:
            break  # Abbiamo raggiunto l'inizio
        curr = parent[curr]
        
    # Se il loop si è interrotto perché curr è None, ma non abbiamo trovato
    # la sorgente, significa che non c'è un percorso (gestito dal controllo inf).
    if path[-1] != source:
        return None, np.inf 

    # Il percorso è costruito all'indietro (da target a source),
    # quindi dobbiamo invertirlo.
    return path[::-1], dist[target]

In [40]:
import heapq 
import numpy as np
from typing import Tuple, List, Optional, Union

def greedy_solver(
    problem: np.ndarray, 
    heuristic_map: np.ndarray, # La matrice (size, 2) delle coordinate
    source: int, 
    target: int
) -> Tuple[Optional[List[int]], Union[float, int]]:
    """
    Trova un percorso (non ottimale) usando Greedy Best-First Search.
    Veloce ma non preciso.
    """
    num_nodes = len(problem)
    
    # Calcola l'euristica (distanza in linea d'aria) dal nodo 'n' al 'target'
    def h(n):
        return np.sqrt(
            np.square(heuristic_map[n, 0] - heuristic_map[target, 0]) + 
            np.square(heuristic_map[n, 1] - heuristic_map[target, 1])
        )

    # Coda di priorità (min-heap)
    # Formato: (h_score, g_score, current_node, path_list)
    # Priorità 1: h_score (euristica) - Vogliamo la più bassa!
    # Priorità 2: g_score (costo reale) - In caso di parità, preferisce il più economico
    
    start_h = h(source)
    pq = [(start_h, 0, source, [source])]  # (h, g, nodo, percorso)
    
    # Usiamo un set per non visitare più volte lo stesso nodo
    # Questo previene i cicli infiniti
    visited = set()

    while pq:
        h_score, g_score, current_node, path = heapq.heappop(pq)
        
        # Trovato!
        if current_node == target:
            return path, g_score
            
        # Evita cicli e lavoro ridondante
        if current_node in visited:
            continue
        visited.add(current_node)
        
        # Esplora i vicini
        for neighbor in range(num_nodes):
            cost = problem[current_node, neighbor]
            
            # Se esiste un arco e non l'ho già visitato
            if cost != np.inf and neighbor not in visited:
                new_g = g_score + cost
                new_h = h(neighbor)
                new_path = path + [neighbor]
                
                # Metti in coda con la priorità h(n)
                heapq.heappush(pq, (new_h, new_g, neighbor, new_path))

    # Se il loop finisce, non c'è un percorso
    return None, np.inf

In [ ]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        #path = nx.shortest_path(G, s, d, weight='weight')
        path = nx.bellman_ford_path(G, s, d, weight='weight') 
        cost = cost = nx.path_weight(G, path, weight='weight')
        ic(d)
    except nx.NetworkXNoPath:
        # Nodes are not connected
        path = None
        cost = np.inf
        path1 = None
        cost1 = np.inf
    except nx.NetworkXUnbounded:
        # Negative cycle detected
        path = None
        cost = -np.inf
        path1 = None
        cost1 = -np.inf
None

ic| 

d: 1
ic| d: 2
ic| d: 3
ic| d: 4
ic| d: 5
ic| d: 6
ic| d: 7
ic| d: 8
ic| d: 9
ic| d: 10
ic| d: 11
ic| d: 12
ic| d: 13
ic| d: 14
ic| d: 15
ic| d: 16
ic| d: 17
ic| d: 18
ic| d: 19
ic| d: 20
ic| d: 21
ic| d: 22
ic| d: 23
ic| d: 24
ic| d: 25
ic| d: 26
ic| d: 27
ic| d: 28
ic| d: 29
ic| d: 30
ic| d: 31
ic| d: 32
ic| d: 33
ic| d: 34
ic| d: 35
ic| d: 36
ic| d: 37
ic| d: 38
ic| d: 39
ic| d: 40
ic| d: 41
ic| d: 42
ic| d: 43
ic| d: 44
ic| d: 45
ic| d: 46
ic| d: 47
ic| d: 48
ic| d: 49
ic| d: 50
ic| d: 51
ic| d: 52
ic| d: 53
ic| d: 54
ic| d: 55
ic| d: 56
ic| d: 57
ic| d: 58
ic| d: 59
ic| d: 60
ic| d

KeyboardInterrupt: 

In [43]:
for s, d in combinations(range(problem.shape[0]), 2):
    
    try:
        #path = nx.shortest_path(G, s, d, weight='weight')
        heuristic_map = np.random.rand(problem.shape[0], 2)
        path, cost = greedy_solver(problem, heuristic_map, s, d)
        #ic(s, d, cost,  cost)
        ic(d)
    except nx.NetworkXNoPath:
        # Nodes are not connected
        path = None
        cost = np.inf
    except nx.NetworkXUnbounded:
        # Negative cycle detected
        path = None
        cost = -np.inf
None

ic| d: 1
ic| d: 2
ic| d: 3
ic| d: 4
ic| d: 5
ic| d: 6
ic| d: 7
ic| d: 8
ic| d: 9
ic| d: 10
ic| d: 11
ic| d: 12
ic| d: 13
ic| d: 14
ic| d: 15
ic| d: 16
ic| d: 17
ic| d: 18
ic| d: 19
ic| d: 20


KeyboardInterrupt: 